# 6.3 Geographic Visualization

#### 01. Import libraries and Data
#### 02. Plotting a choropleth
#### 03. Discussing the result
#### 04. Export Data

## 01. Importing Libraries and Data

In [3]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json
from folium import plugins

In [4]:
# defing path
path = r'/Users/Ayaz/Desktop/Boat Sales Analysis'

In [6]:
%matplotlib inline

In [5]:
# Importing new cleaned dataframe for spitial analysis.
df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'df_boats_countriesclean.csv'), index_col = False)

In [7]:
# Display all rows without scrolling.
pd.set_option('display.max_rows', None)

## 02. Plotting a Choropleth

In [8]:
# Import ".json" file for the entire globe.
country_geo = r'/Users/Ayaz/Desktop/Boat Sales Analysis/02 Data/Original Data/countries.geo.json'

In [9]:
# To look at the JSON file contents.
f = open(r'/Users/Ayaz/Desktop/Boat Sales Analysis/02 Data/Original Data/countries.geo.json',)

# returns JSON object asa dictionary
data = json.load(f)
  
# Iterating through the json list
for i in data['features']:
    print(i)

{'type': 'Feature', 'id': 'AFG', 'properties': {'name': 'Afghanistan'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[61.210817, 35.650072], [62.230651, 35.270664], [62.984662, 35.404041], [63.193538, 35.857166], [63.982896, 36.007957], [64.546479, 36.312073], [64.746105, 37.111818], [65.588948, 37.305217], [65.745631, 37.661164], [66.217385, 37.39379], [66.518607, 37.362784], [67.075782, 37.356144], [67.83, 37.144994], [68.135562, 37.023115], [68.859446, 37.344336], [69.196273, 37.151144], [69.518785, 37.608997], [70.116578, 37.588223], [70.270574, 37.735165], [70.376304, 38.138396], [70.806821, 38.486282], [71.348131, 38.258905], [71.239404, 37.953265], [71.541918, 37.905774], [71.448693, 37.065645], [71.844638, 36.738171], [72.193041, 36.948288], [72.63689, 37.047558], [73.260056, 37.495257], [73.948696, 37.421566], [74.980002, 37.41999], [75.158028, 37.133031], [74.575893, 37.020841], [74.067552, 36.836176], [72.920025, 36.720007], [71.846292, 36.509942], [71.262348, 36.074388]

In [10]:
df.head()

Unnamed: 0            boat_type          manufacturer                 type  \
0           0          Motor Yacht  Rigiflex power boats  new boat from stock   
1           1  Center console boat     Terhi power boats  new boat from stock   
2           3           Sport Boat    Pioner power boats  new boat from stock   
3           4         Fishing Boat    Linder power boats  new boat from stock   
4           6            Catamaran               unknown   Used boat,Unleaded   

   year_built  length  width       material  views_last_7_days currency  \
0        2017    4.00   1.90        unknown                226      CHF   
1        2020    4.00   1.50  Thermoplastic                 75      EUR   
2        2020    3.00   1.00        unknown                 64      DKK   
3        2019    3.55   1.46      Aluminium                 58      EUR   
4        1999    6.20   2.38      Aluminium                474      CHF   

   value  value_in_eur      country  
0   3337        3337.0  Switzerland  
1   3490        3490.0      Germany  
2  25900        3367.0      Denmark  
3   3399        3399.0      Germany  
4   3600        3600.0  Switzerland

In [11]:
# Creating subset called 'df_choropleth' from columns 'country' and 'views last 7 days'
df_choropleth = df.groupby('country')['views_last_7_days'].sum()

In [12]:
# Create a Choropleth Geographic Visualization for number of views per country

df_choropleth = df.groupby('country')['views_last_7_days'].sum()
df_choropleth=pd.DataFrame({'country':df_choropleth.index, 'views_last_7_days':df_choropleth.values})
state_data_indexed = df_choropleth.set_index('country')
state_data_indexed['views_last_7_days'] = state_data_indexed['views_last_7_days'].astype(np.float64)

map = folium.Map(location = [49,2.2], zoom_start = 2)

cp=folium.Choropleth(
    geo_data = country_geo, 
    data = df_choropleth,
    columns = ['country', 'views_last_7_days'],
    key_on = 'feature.properties.name', 
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "views").add_to(map)
folium.LayerControl().add_to(map)

for s in cp.geojson.data['features']:
    if s['properties']['name'] in state_data_indexed.index:
        s['properties']['views']=state_data_indexed.loc[s['properties']['name'],'views_last_7_days']
    else:
        s['properties']['views']=0    
        
folium.GeoJsonTooltip(['name','views']).add_to(cp.geojson)

map

## 03. Discussion

#### Questions to be Explore:
1.	Which regions in Europe are boats looked at?
2.	characteristics of the most-viewed boat listings in the last 7 days.
3.	Is it the most expensive boat that gets the most views?
4.	Are there common features among the most-viewed boats?

#### Findings 

This choropleth graph does provide an answer to to the question of which regions in Europe have the most boat views. In other words, does a large proportion of internet traffic originate in European regions?
This map provides a satisfactory answer to this question. Looking at the map, we can observe that the majority of views or traffic on the boat sales website came from the European regions.  Switzerland had the most views per year, around 345950, followed by Germany with 254349 and Italy with 208634, while Lebanon had the least traffic on the website, with 47 views, Venezuela with 64 views, and Egypt with just 81 views on the boats sales website. 



This insight leads to some new questions.

#### What are the characteristics of the most-viewed boats?

a) Which type of boat gets the most views on average?

b) Which type of boat engine gets more views?

c) Which boat material receives the most attention?

d) From which country do used or new boats receive the most views?

## 04. Export the Data

In [13]:
# Exporting new country cleaned dataframe as csv file.
df.to_csv(os.path.join(path, '02 Data','Prepared Data', 'df_boats_sales.csv'))